In [1]:
import csv
import cv2
import numpy as np
import os

from sklearn.model_selection import train_test_split
import sklearn

from keras.models import Sequential
from keras.layers import Dense, Flatten, Lambda, Convolution2D, Cropping2D

Using TensorFlow backend.


In [2]:
base_path = '/hdd/datasets/udacity_sim_data/recorded_data/'

In [3]:
samples = []
with open(base_path + 'train/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)
with open(base_path + 'recovery/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)

In [4]:
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

In [5]:
c_factor = 0.3

def process_image(sample, angle, correction=0.0, flip=False):
    img = cv2.imread(sample)
    corrected_angle = angle + correction
    if flip:
        return np.fliplr(img), -corrected_angle
    else:
        return img, corrected_angle

def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        sklearn.utils.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                angle = float(batch_sample[3])
                
                # center
                processed_image, processed_angle = process_image(batch_sample[0], angle)
                images.append(processed_image)
                angles.append(processed_angle)
                processed_image, processed_angle = process_image(batch_sample[0], angle, flip=True)
                images.append(processed_image)
                angles.append(processed_angle)

                # left
                processed_image, processed_angle = process_image(batch_sample[1], angle, correction=c_factor)
                images.append(processed_image)
                angles.append(processed_angle)
                processed_image, processed_angle = process_image(batch_sample[1], angle, correction=c_factor, flip=True)
                images.append(processed_image)
                angles.append(processed_angle)

                # right
                processed_image, processed_angle = process_image(batch_sample[2], angle, correction=-c_factor)
                images.append(processed_image)
                angles.append(processed_angle)
                processed_image, processed_angle = process_image(batch_sample[2], angle, correction=-c_factor, flip=True)
                images.append(processed_image)
                angles.append(processed_angle)

            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

In [6]:
input_shape=(160,320,3)

model = Sequential()
model.add(Cropping2D(cropping=((70,25), (0,0)), input_shape=input_shape))
model.add(Lambda(lambda x: (x / 255.0) - 0.5))
model.add(Convolution2D(24, 5, 5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(36, 5, 5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(48, 5, 5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [9]:
model.compile(loss='mse', optimizer='adam')

In [10]:
model.fit_generator(train_generator, 
                    samples_per_epoch= len(train_samples), 
                    validation_data=validation_generator,
                    nb_val_samples=len(validation_samples), 
                    nb_epoch=6)
model.save('model.h5')

Epoch 1/6
3072/3212 [===========================>..] - ETA: 0s - loss: 0.0485

/home/ryan/.conda/envs/dl35/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


3264/3212 [==============================] - 6s - loss: 0.0487 - val_loss: 0.0456
Epoch 2/6
3264/3212 [==============================] - 5s - loss: 0.0386 - val_loss: 0.0378
Epoch 3/6
3264/3212 [==============================] - 5s - loss: 0.0328 - val_loss: 0.0405
Epoch 4/6
3264/3212 [==============================] - 5s - loss: 0.0364 - val_loss: 0.0409
Epoch 5/6
3264/3212 [==============================] - 5s - loss: 0.0325 - val_loss: 0.0352
Epoch 6/6
3336/3212 [===============================] - 5s - loss: 0.0337 - val_loss: 0.0313
